# 读取mongodb数据库文件

In [1]:
import pymongo
import pandas as pd

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["Macao"] 
tb = db['Macao_PT_2019-08-01_10:00']

# x = [i for i in tb.find()]
x = [i for i in tb.find().limit(5).skip(5)] # 数据较大时使用
df = pd.DataFrame(x)

In [3]:
df

,_id,all_distance,all_time,all_price,all_walk_distance,all_walk_time,access_walk_distance,egress_walk_distance,number_of_boardings,label,pt
0,62b01ceee7210870c6afca65,2808,1455,6.0,645,645,499,84,1,ID704547,1
1,62b01ceee7210870c6afca6a,1964,1254,6.0,465,465,231,202,1,ID581966,1
2,62b01ceee7210870c6afca6b,5213,2225,6.0,584,584,194,305,1,ID1199860,1
3,62b01ceee7210870c6afca6c,1543,1424,6.0,789,789,295,467,1,ID1017821,1
4,62b01ceee7210870c6afca6d,3116,1903,6.0,689,689,257,222,1,ID1183318,1


# geopandas创建点线面

In [4]:
import geopandas as gpd
import pandas as pd

## 准备一些数据

In [5]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

db = myclient["fang_shanghai"] 
tb = db['data']
x = [i for i in tb.find({},{"projname": 1, "x": 1,"y": 1}).limit(5).skip(5)] # 数据较大时使用
df_point = pd.DataFrame(x)

db = myclient["Traffic"] 
tb = db['2020-06-03-19-59']
x = [i for i in tb.find({},{"direction": 1, "line_wgs84": 1}).limit(5).skip(5)] # 数据较大时使用
df_line = pd.DataFrame(x)

# x = [i for i in tb.find({},{"direction": 1, "line_wgs84": 1}).limit(5).skip(5)] # 数据较大时使用
# df_line = pd.DataFrame(x)

In [6]:
df_point

,_id,projname,x,y
0,607e40479743483142d2ee45,上海阳城,121.40511322021484,31.127880096435547
1,607e40479743483142d2ee46,春申景城一期,121.42086029052734,31.110912322998047
2,607e40479743483142d2ee47,长桥四村,121.44601440429688,31.139463424682617
3,607e40479743483142d2ee48,上海春城,121.4154052734375,31.109514236450195
4,607e40479743483142d2ee49,仁恒滨江园,121.51408386230469,31.234397888183594


In [7]:
geomtery = gpd.points_from_xy(df_point['x'],df_point['y'],crs='epsg:4326')
points = gpd.GeoDataFrame(df_point,geometry=geomtery)

In [8]:
points

,_id,projname,x,y,geometry
0,607e40479743483142d2ee45,上海阳城,121.40511322021484,31.127880096435547,POINT (121.40511 31.12788)
1,607e40479743483142d2ee46,春申景城一期,121.42086029052734,31.110912322998047,POINT (121.42086 31.11091)
2,607e40479743483142d2ee47,长桥四村,121.44601440429688,31.139463424682617,POINT (121.44601 31.13946)
3,607e40479743483142d2ee48,上海春城,121.4154052734375,31.109514236450195,POINT (121.41541 31.10951)
4,607e40479743483142d2ee49,仁恒滨江园,121.51408386230469,31.234397888183594,POINT (121.51408 31.23440)


In [9]:
df_line

,_id,direction,line_wgs84
0,5ed790b16df18b3ccb3ce5e2,从苏州市东边界到苏州市边界,"[[121.15059750532379, 31.280073883197833], [12..."
1,5ed790b16df18b3ccb3ce5e3,从胜浦路到界浦路,"[[120.82776544305959, 31.315833264040876], [12..."
2,5ed790b16df18b3ccb3ce5e4,从界浦路到唯胜路,"[[120.84966717369905, 31.320795084011507], [12..."
3,5ed790b16df18b3ccb3ce5e5,从界浦路南口到强胜路,"[[120.85361187963333, 31.312110897248864], [12..."
4,5ed790b16df18b3ccb3ce5e6,从强胜路到界浦路南口,"[[120.84262633429745, 31.338606660108773], [12..."


In [10]:
from shapely.geometry import LineString

geomtery = gpd.GeoSeries([LineString(i) for i in df_line['line_wgs84']],crs='epsg:4326')

lines = gpd.GeoDataFrame(df_line,
                         geometry=geomtery,
                         crs='epsg:4326')

In [11]:
lines

,_id,direction,line_wgs84,geometry
0,5ed790b16df18b3ccb3ce5e2,从苏州市东边界到苏州市边界,"[[121.15059750532379, 31.280073883197833], [12...","LINESTRING (121.15060 31.28007, 121.14976 31.2..."
1,5ed790b16df18b3ccb3ce5e3,从胜浦路到界浦路,"[[120.82776544305959, 31.315833264040876], [12...","LINESTRING (120.82777 31.31583, 120.83050 31.3..."
2,5ed790b16df18b3ccb3ce5e4,从界浦路到唯胜路,"[[120.84966717369905, 31.320795084011507], [12...","LINESTRING (120.84967 31.32080, 120.84961 31.3..."
3,5ed790b16df18b3ccb3ce5e5,从界浦路南口到强胜路,"[[120.85361187963333, 31.312110897248864], [12...","LINESTRING (120.85361 31.31211, 120.85360 31.3..."
4,5ed790b16df18b3ccb3ce5e6,从强胜路到界浦路南口,"[[120.84262633429745, 31.338606660108773], [12...","LINESTRING (120.84263 31.33861, 120.84264 31.3..."
